In [1]:
import os
import wget
import zipfile
import openai
import json  
import wget
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv()
# Config for Azure OpenAI.
OPENAI_API_TYPE = "azure"
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-07-01-preview"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
import pandas as pd

df = pd.read_csv('../data/black_friday_sales.csv')

In [4]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


### CSV Agent

NOTE: this agent calls the Pandas DataFrame agent under the hood, which in turn calls the Python agent, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.



In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI


In [ ]:
llm4 = AzureChatOpenAI(
    openai_api_base= OPENAI_API_BASE,
    openai_api_version= OPENAI_API_VERSION,
    deployment_name="gpt-4-32k",
    temperature=0,
    openai_api_key= OPENAI_API_KEY ,
    openai_api_type = OPENAI_API_TYPE,
)

In [16]:
model = AzureOpenAI(
    openai_api_base= OPENAI_API_BASE,
    openai_api_version= OPENAI_API_VERSION,
    deployment_name="gpt-35-turbo-instruct",
    temperature=0,
    openai_api_key= OPENAI_API_KEY,
    openai_api_type = OPENAI_API_TYPE,
)
model("tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [28]:
agent = create_csv_agent(
        llm4,
        "../data/black_friday_sales.csv", 
        verbose=True)

In [32]:
agent

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], partial_variables={'df_head': '|    |   User_ID | Product_ID   | Gender   | Age   |   Occupation | City_Category   | Stay_In_Current_City_Years   |   Marital_Status |   Product_Category_1 |   Product_Category_2 |   Product_Category_3 |   Purchase |\n|---:|----------:|:-------------|:---------|:------|-------------:|:----------------|:-----------------------------|-----------------:|---------------------:|---------------------:|---------------------:|-----------:|\n|  0 |   1000001 | P00069042    | F        | 0-17  |           10 | A               | 2                            |                0 |                    3 |                  nan |                  nan |       8370 |\n|  1 |   1000001 | P00248942    | F        | 0-17  |           10 | A               | 2                            |                0 |                    1 |                  

In [33]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

You are working with a pandas dataframe in Python. The name of the dataframe is `df`.

You should use the tools below to answer the question posed of you:

`python_repl_ast:` A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [python_repl_ast]

Action Input: the input to the action

Observation: the result of the action

(this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

This is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [34]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find out the number of rows in a dataframe, we can use the `shape` attribute of the dataframe. The `shape` attribute returns a tuple representing the dimensionality of the DataFrame where the first element of the tuple is the number of rows and the second element is the number of columns.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 550068
Thought:I now know the final answer
Final Answer: There are 550068 rows in the dataframe.

> Finished chain.


'There are 550068 rows in the dataframe.'

In [35]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: To find out how many people are female, I need to count the number of rows where the 'Gender' column is 'F'.
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].shape[0]
Observation: 135809
Thought:I now know the final answer.
Final Answer: There are 135809 females.

> Finished chain.


'There are 135809 females.'

In [36]:
agent.run("how many people have stayed more than 3 years in the city?")



> Entering new AgentExecutor chain...
Thought: To find out how many people have stayed more than 3 years in the city, I need to count the number of rows where the 'Stay_In_Current_City_Years' column is '4+'. 
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] == '4+'].shape[0]
Observation: 84726
Thought:I now know the final answer
Final Answer: 84726 people have stayed more than 3 years in the city.

> Finished chain.


'84726 people have stayed more than 3 years in the city.'

In [37]:
agent.run("how many people have stayed more than 3 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to include only rows where the 'Stay_In_Current_City_Years' column is '4+' (which indicates more than 3 years) and the 'Gender' column is 'F' (which indicates female). Then, I can count the number of such rows.
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] == '4+') & (df['Gender'] == 'F')].shape[0]
Observation: 18596
Thought:I now know the final answer.
Final Answer: There are 18596 females who have stayed more than 3 years in the city.

> Finished chain.


'There are 18596 females who have stayed more than 3 years in the city.'

In [38]:
agent.run("Are there more males or females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females in the dataframe. The 'Gender' column contains this information.
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: Gender
M    414259
F    135809
Name: count, dtype: int64
Thought:I now know the final answer
Final Answer: There are more males than females.

> Finished chain.


'There are more males than females.'

In [29]:
agent.run("What are the column names?")



> Entering new AgentExecutor chain...
Thought: To get the column names of a pandas dataframe, we can use the `.columns` attribute.
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')
Thought:I now know the column names of the dataframe.
Final Answer: The column names are 'User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', and 'Purchase'.

> Finished chain.


"The column names are 'User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', and 'Purchase'."

In [30]:
agent.run("What is the average age?")



> Entering new AgentExecutor chain...
Thought: The 'Age' column in the dataframe is not numeric, but categorical. Therefore, it's not possible to calculate an average age directly from this data. However, we can calculate the most frequent age category, which could give an indication of the average age group. 
Action: python_repl_ast
Action Input: df['Age'].mode()[0]
Observation: 26-35
Thought:The most frequent age category in the dataframe is '26-35'.
Final Answer: The most frequent age category is '26-35'.

> Finished chain.


"The most frequent age category is '26-35'."

In [39]:
agent.run("Which country appears the most and how many times does it appear?")



> Entering new AgentExecutor chain...
Thought: The dataframe does not contain a column for country. Therefore, it is not possible to answer the question with the given data. 
Final Answer: The dataframe does not contain information about countries.

> Finished chain.


'The dataframe does not contain information about countries.'

### Multi CSV Example

In [ ]:
agent = create_csv_agent(
    llm4,
    ["titanic.csv", "titanic_age_fillna.csv"],
    verbose=True,
)
agent.run("how many rows in the age column are different between the two dfs?")